# Forward applications

By David Ortiz and Rodrigo Salas, 2024

## Activity overview

In this activity, 

## Activity goals

By the end of this activity, you should be able to:

 - understand the need of numerical solution to complex models
 - understand the advantages of using Physics-Informed Neural Networks (PINNs) compared to common Artificial Neural Networks (ANN)
 - train data-driven PINNs using PyTorch
 - solve non-linear models using PINNs


## Mathematical description of the problem

$$
\begin{alignat*}{3}
    \text{PDE:} \quad & \frac{\partial y}{\partial t} &&= \kappa\frac{\partial^2 y}{\partial x^2} - e^{-t}(\sin(\pi x) - \pi^2 \kappa \sin(\pi x)), \quad && x \in [-1, 1], \quad t \in [0, 1]. \quad \kappa\in\R \\
\end{alignat*}
$$


$$
\begin{alignat*}{3}
    \text{PDE:} \quad & \frac{\partial y}{\partial t} &&= \kappa\frac{\partial^2 y}{\partial x^2} - e^{-t}(\sin(\pi x) - \pi^2 \kappa \sin(\pi x)), \quad && x \in [-1, 1], \quad t \in [0, 1]. \quad \kappa\in\R \\
    \text{IC:} \quad & y(x, 0) &&= \sin(\pi x), && x \in [-1, 1] \\
    \text{BC:} \quad & y(-1, t)&& = y(1, t)= 0, && t \in [0, 1] \\
    \text{Solution:} \quad & y(x, t) &&= e^{-t} \sin(\pi x)
\end{alignat*}
$$

## Initial setup

We begin by importing some usefull packages, and defining some functions

In [ ]:
# Import NumPy for numerical operations
import numpy as np
# Import PyTorch for building and training neural networks
import torch
import torch.nn as nn
import torch.optim as optim
# Import Matplotlib for plotting
import matplotlib.pyplot as plt
# Import a utility module 
import utils 
# Import the time module to time our training process
import time
# Ignore Warning Messages
import warnings
warnings.filterwarnings("ignore")